# Standalone Python scripts

*(micro-input)*

## Use-cases

- Use your own code on various machines
- Provide others with your scripts, exactly as they are

## Script creation with `uv`

- Add dummy hashbang, e.g., `#! dummy`
- `uv init --script calculate-something.py`
- Insert suggested hashbang `#!/usr/bin/env -S uv run --script`
- Add dependencies `uv add --script calculate-something.py numpy pandas statsmodels`
- Allow execution `chmod u+x calculate-something.py`
- Run it `./calculate-something.py`

```python
#!/usr/bin/env -S uv run --script
#
# /// script
# requires-python = ">=3.11"
# dependencies = [
#     "analysis",
#     "boku-met-gis-utils",
#     "dask[distributed]>=2025.12.0",
#     "data-utils",
#     "numpy>=2.3.5",
#     "xarray[accel,io,parallel]>=2025.12.0",
# ]
#
# [tool.uv.sources]
# data-utils = { path = "../../data_utils", editable = true }
# boku-met-gis-utils = { git = "https://github.com/boku-met/gis-utils" }
# analysis = { path = "../", editable = true }
# ///


from analysis import xr_fit_time_trend
from data_utils import interm_path
from dask.distributed import Client, LocalCluster
from pathlib import Path
import xarray as xr
import numpy as np


if __name__ == "__main__":
    for model in ["ICON", "IFS-NEMO"]:
        results_path = Path(f"../data/soil_water_depletion__log-scaled_trend_fit_results__soybean__{model}.nc")
        if results_path.exists():
            print(f"Results for model {model} are present.")
            continue
        water = xr.concat(
            [
                xr.open_dataset(interm_path / f"{model}_HIST__overhaul" / "water_ds.zarr", chunks={}),
                xr.open_dataset(interm_path / f"{model}_SSP3-7.0__overhaul" / "water_ds.zarr", chunks={}),
            ],
            "time",
        ).soil_depletion.sel(crop="soybean")
        with LocalCluster(n_workers=24, dashboard_address=":44444") as cluster, Client(cluster) as client:
            water.resample(time="YE").sum().pipe(np.log).pipe(xr_fit_time_trend).to_netcdf(results_path)
```